In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm
import matplotlib.pyplot as plt
import scipy.optimize as optimize
from scipy.stats import norm
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller

import warnings
warnings.filterwarnings("ignore")

In [2]:
import rqdatac as rq
rq.init()

In [3]:
index_weight = pd.read_json("index_weight.json")[0]
index_weight

PS    0.472150
V     0.139169
SF    0.154166
SA    0.060324
LC    0.132901
JM   -0.018898
MA    0.065181
AO   -0.007069
SP    0.059142
RU   -0.057066
Name: 0, dtype: float64

In [4]:
si_data = rq.futures.get_dominant_price("SI", frequency="1m").loc["SI"]
si_data

,trading_date,dominant_id,open,close,high,low,total_turnover,volume,open_interest
datetime,,,,,,,,,
2025-06-24 09:01:00,2025-06-24,SI2509,7420.0,7420.0,7430.0,7355.0,0,20857.0,302388.0
2025-06-24 09:02:00,2025-06-24,SI2509,7420.0,7435.0,7445.0,7410.0,0,7337.0,302936.0
2025-06-24 09:03:00,2025-06-24,SI2509,7435.0,7420.0,7450.0,7420.0,0,6267.0,302572.0
2025-06-24 09:04:00,2025-06-24,SI2509,7420.0,7415.0,7435.0,7400.0,0,5614.0,302818.0
2025-06-24 09:05:00,2025-06-24,SI2509,7415.0,7415.0,7425.0,7405.0,0,6241.0,303889.0
...,...,...,...,...,...,...,...,...,...
2025-09-24 14:46:00,2025-09-24,SI2511,9010.0,9010.0,9020.0,9010.0,0,1759.0,276821.0
2025-09-24 14:47:00,2025-09-24,SI2511,9015.0,9005.0,9015.0,9005.0,0,559.0,276665.0
2025-09-24 14:48:00,2025-09-24,SI2511,9010.0,9005.0,9015.0,9005.0,0,934.0,276662.0


In [5]:
def preprocess(data: tuple):

    minute_price = data["close"]
    minute_price.loc[data["trading_date"].iloc[0] + pd.Timedelta(hours=9)] = data["open"].iloc[0]
    minute_price = minute_price.sort_index()
    trading_date = minute_price.index.strftime("%Y-%m-%d")
    mask = (minute_price.index >= pd.to_datetime(trading_date + " 09:00:00")) & \
           (minute_price.index <= pd.to_datetime(trading_date + " 15:00:00"))
    minute_price = minute_price[mask]

    return minute_price

In [17]:
def preprocess_vwap(data: tuple):

    minute_vwap = (data["close"] * data["volume"]).cumsum() / data["volume"].cumsum()
    trading_date = minute_vwap.index.strftime("%Y-%m-%d")
    mask = (minute_vwap.index >= pd.to_datetime(trading_date + " 09:00:00")) & \
           (minute_vwap.index <= pd.to_datetime(trading_date + " 15:00:00"))
    minute_vwap = minute_vwap[mask]
    minute_vwap = minute_vwap / minute_vwap.iloc[0]

    return minute_vwap

In [7]:
si_minute = si_data.groupby("trading_date").apply(preprocess)
si_minute.name = "SI"
si_val = si_minute.groupby(level=0).apply(lambda x: (((x.diff() / x.shift()).fillna(0) + 1).cumprod())).droplevel(0)
si_val

trading_date  datetime           
2025-06-24    2025-06-24 09:00:00    1.000000
              2025-06-24 09:01:00    1.000000
              2025-06-24 09:02:00    1.002022
              2025-06-24 09:03:00    1.000000
              2025-06-24 09:04:00    0.999326
                                       ...   
2025-09-24    2025-09-24 14:46:00    1.006704
              2025-09-24 14:47:00    1.006145
              2025-09-24 14:48:00    1.006145
              2025-09-24 14:49:00    1.007263
              2025-09-24 14:50:00    1.008939
Name: SI, Length: 15132, dtype: float64

In [18]:
si_vwap = si_data.groupby("trading_date").apply(preprocess_vwap)
si_vwap

trading_date  datetime           
2025-06-24    2025-06-24 09:01:00    1.000000
              2025-06-24 09:02:00    1.000526
              2025-06-24 09:03:00    1.000430
              2025-06-24 09:04:00    1.000276
              2025-06-24 09:05:00    1.000148
                                       ...   
2025-09-24    2025-09-24 14:46:00    1.012258
              2025-09-24 14:47:00    1.012261
              2025-09-24 14:48:00    1.012266
              2025-09-24 14:49:00    1.012270
              2025-09-24 14:50:00    1.012283
Length: 15065, dtype: float64

In [9]:
all_minute = pd.DataFrame()
for underlying in index_weight.index:
    data = rq.futures.get_dominant_price(underlying, frequency="1m").loc[underlying]
    minute = data.groupby("trading_date").apply(preprocess)
    minute.name = underlying
    all_minute = pd.concat([all_minute, minute], axis=1)

all_minute

PS       V      SF      SA       LC  \
2025-06-24 2025-06-24 09:00:00  30605.0  5027.0  5466.0  1266.0  59100.0   
           2025-06-24 09:01:00  30560.0  5003.0  5470.0  1262.0  59300.0   
           2025-06-24 09:02:00  30700.0  4988.0  5466.0  1262.0  59520.0   
           2025-06-24 09:03:00  30780.0  4984.0  5472.0  1261.0  59240.0   
           2025-06-24 09:04:00  30850.0  4985.0  5472.0  1262.0  59380.0   
...                                 ...     ...     ...     ...      ...   
2025-09-24 2025-09-24 14:46:00  51200.0  4921.0  5758.0  1302.0  73200.0   
           2025-09-24 14:47:00  51180.0  4918.0  5754.0  1303.0  73160.0   
           2025-09-24 14:48:00  51185.0  4918.0  5758.0  1303.0  73100.0   
           2025-09-24 14:49:00  51200.0  4921.0  5760.0  1304.0  73180.0   
           2025-09-24 14:50:00  51245.0  4923.0  5762.0  1305.0  73180.0   

                                    JM      MA      AO      SP       RU  
2025-06-24 2025-06-24 09:00:00   922.0  2584.0  2954.0  5266.0  14815.0  
           2025-06-24 09:01:00   913.5  2472.0  2958.0  5156.0  14625.0  
           2025-06-24 09:02:00   916.5  2464.0  2961.0  5152.0  14650.0  
           2025-06-24 09:03:00   913.5  2468.0  2964.0  5146.0  14610.0  
           2025-06-24 09:04:00   915.5  2470.0  2960.0  5152.0  14625.0  
...                                ...     ...     ...     ...      ...  
2025-09-24 2025-09-24 14:46:00  1223.5  2353.0  2905.0  5022.0  15600.0  
           2025-09-24 14:47:00  1224.0  2352.0  2905.0  5022.0  15590.0  
           2025-09-24 14:48:00  1223.5  2353.0  2905.0  5022.0  15585.0  
           2025-09-24 14:49:00  1225.0  2354.0  2907.0  5028.0  15590.0  
           2025-09-24 14:50:00  1225.0  2354.0  2906.0  5026.0  15585.0  

[15132 rows x 10 columns]

In [10]:
index_val = all_minute.groupby(level=0).apply(lambda x: (((x.diff() / x.shift()).apply(lambda y: y.dot(index_weight), axis=1)).fillna(0) + 1).cumprod()).droplevel(0)
index_val

2025-06-24  2025-06-24 09:00:00    1.000000
            2025-06-24 09:01:00    0.995849
            2025-06-24 09:02:00    0.997545
            2025-06-24 09:03:00    0.998405
            2025-06-24 09:04:00    0.999898
                                     ...   
2025-09-24  2025-09-24 14:46:00    1.010251
            2025-09-24 14:47:00    1.009845
            2025-09-24 14:48:00    1.009944
            2025-09-24 14:49:00    1.010470
            2025-09-24 14:50:00    1.011044
Length: 15132, dtype: float64

In [11]:
val_diff = index_val - si_val
counts, bin_edges = np.histogram(val_diff, bins=100)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

def normal_func(x, mu, sigma, amplitude):
    return amplitude * norm.pdf(x, mu, sigma)

# 使用曲线拟合
params, params_covariance = optimize.curve_fit(
    normal_func, bin_centers, counts, p0=[val_diff.mean(), val_diff.std(), counts.max()]
)

print(f"拟合参数: μ={params[0]:.6f}, σ={params[1]:.6f}, 幅度={params[2]:.6f}")

拟合参数: μ=0.001255, σ=0.009867, 幅度=12.968318


In [12]:
confidence_levels = [0.10, 0.20, 0.30]  # 10%, 20%, 30%水平

print("标准正态分布的单边分位数:")
for level in confidence_levels:
    z_score = norm.ppf(1 - level)  # ppf是分位数函数
    print(f"{int(level*100)}%水平: μ + {z_score:.4f}σ")

标准正态分布的单边分位数:
10%水平: μ + 1.2816σ
20%水平: μ + 0.8416σ
30%水平: μ + 0.5244σ


In [13]:
si_val.name = "si"
index_val.name = "index"
val_diff.name = "spread"

data = pd.concat([val_diff, si_val, index_val], axis=1)
data

spread        si     index
2025-06-24 2025-06-24 09:00:00  0.000000  1.000000  1.000000
           2025-06-24 09:01:00 -0.004151  1.000000  0.995849
           2025-06-24 09:02:00 -0.004477  1.002022  0.997545
           2025-06-24 09:03:00 -0.001595  1.000000  0.998405
           2025-06-24 09:04:00  0.000571  0.999326  0.999898
...                                  ...       ...       ...
2025-09-24 2025-09-24 14:46:00  0.003547  1.006704  1.010251
           2025-09-24 14:47:00  0.003700  1.006145  1.009845
           2025-09-24 14:48:00  0.003799  1.006145  1.009944
           2025-09-24 14:49:00  0.003207  1.007263  1.010470
           2025-09-24 14:50:00  0.002106  1.008939  1.011044

[15132 rows x 3 columns]

In [14]:
def add_lag(data: pd.DataFrame):
    for lag in range(1, 11):
        data["si_lag_%d" % lag] = data["si"].shift(lag)
        data["index_lag_%d" % lag] = data["index"].shift(lag)
    return data

lag_data = data.groupby(level=0).apply(add_lag).droplevel(0).dropna(how="any")
lag_data

spread        si     index  si_lag_1  \
2025-06-24 2025-06-24 09:10:00 -0.006857  1.007412  1.000555  1.007412   
           2025-06-24 09:11:00 -0.007177  1.006739  0.999562  1.007412   
           2025-06-24 09:12:00 -0.007726  1.006739  0.999013  1.006739   
           2025-06-24 09:13:00 -0.004925  1.004043  0.999118  1.006739   
           2025-06-24 09:14:00 -0.005919  1.005391  0.999472  1.004043   
...                                  ...       ...       ...       ...   
2025-09-24 2025-09-24 14:46:00  0.003547  1.006704  1.010251  1.007263   
           2025-09-24 14:47:00  0.003700  1.006145  1.009845  1.006704   
           2025-09-24 14:48:00  0.003799  1.006145  1.009944  1.006145   
           2025-09-24 14:49:00  0.003207  1.007263  1.010470  1.006145   
           2025-09-24 14:50:00  0.002106  1.008939  1.011044  1.007263   

                                index_lag_1  si_lag_2  index_lag_2  si_lag_3  \
2025-06-24 2025-06-24 09:10:00     0.999961  1.003369     0.999331  1.002022   
           2025-06-24 09:11:00     1.000555  1.007412     0.999961  1.003369   
           2025-06-24 09:12:00     0.999562  1.007412     1.000555  1.007412   
           2025-06-24 09:13:00     0.999013  1.006739     0.999562  1.007412   
           2025-06-24 09:14:00     0.999118  1.006739     0.999013  1.006739   
...                                     ...       ...          ...       ...   
2025-09-24 2025-09-24 14:46:00     1.009294  1.006704     1.010410  1.006704   
           2025-09-24 14:47:00     1.010251  1.007263     1.009294  1.006704   
           2025-09-24 14:48:00     1.009845  1.006704     1.010251  1.007263   
           2025-09-24 14:49:00     1.009944  1.006145     1.009845  1.006704   
           2025-09-24 14:50:00     1.010470  1.006145     1.009944  1.006145   

                                index_lag_3  si_lag_4  ...  si_lag_6  \
2025-06-24 2025-06-24 09:10:00     0.999219  1.003369  ...  0.999326   
           2025-06-24 09:11:00     0.999331  1.002022  ...  0.999326   
           2025-06-24 09:12:00     0.999961  1.003369  ...  1.003369   
           2025-06-24 09:13:00     1.000555  1.007412  ...  1.002022   
           2025-06-24 09:14:00     0.999562  1.007412  ...  1.003369   
...                                     ...       ...  ...       ...   
2025-09-24 2025-09-24 14:46:00     1.010795  1.007821  ...  1.009497   
           2025-09-24 14:47:00     1.010410  1.006704  ...  1.006704   
           2025-09-24 14:48:00     1.009294  1.006704  ...  1.007821   
           2025-09-24 14:49:00     1.010251  1.007263  ...  1.006704   
           2025-09-24 14:50:00     1.009845  1.006704  ...  1.006704   

                                index_lag_6  si_lag_7  index_lag_7  si_lag_8  \
2025-06-24 2025-06-24 09:10:00     0.999898  1.000000     0.998405  1.002022   
           2025-06-24 09:11:00     0.999478  0.999326     0.999898  1.000000   
           2025-06-24 09:12:00     0.998945  0.999326     0.999478  0.999326   
           2025-06-24 09:13:00     0.999219  1.003369     0.998945  0.999326   
           2025-06-24 09:14:00     0.999331  1.002022     0.999219  1.003369   
...                                     ...       ...          ...       ...   
2025-09-24 2025-09-24 14:46:00     1.011880  1.009497     1.012598  1.009497   
           2025-09-24 14:47:00     1.010338  1.009497     1.011880  1.009497   
           2025-09-24 14:48:00     1.011152  1.006704     1.010338  1.009497   
           2025-09-24 14:49:00     1.010795  1.007821     1.011152  1.006704   
           2025-09-24 14:50:00     1.010410  1.006704     1.010795  1.007821   

                                index_lag_8  si_lag_9  index_lag_9  si_lag_10  \
2025-06-24 2025-06-24 09:10:00     0.997545  1.000000     0.995849   1.000000   
           2025-06-24 09:11:00     0.998405  1.002022     0.997545   1.000000   
           2025-06-24 09:12:00     0.999898  1.000000     0.998405   1.002022   
           2025-06-24 09:13:00    

In [15]:
X = lag_data.drop(["spread", "si", "index"], axis=1)
X = sm.add_constant(X)
y = lag_data["spread"]
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                 spread   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                 8.521e+04
Date:                Wed, 24 Sep 2025   Prob (F-statistic):               0.00
Time:                        14:50:46   Log-Likelihood:                 78367.
No. Observations:               14462   AIC:                        -1.567e+05
Df Residuals:                   14441   BIC:                        -1.565e+05
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const           -0.0006      0.001     -0.773   

In [16]:
result = adfuller(val_diff, autolag="AIC")
result

(-7.8367573099385375,
 6.086096072896393e-12,
 1,
 15130,
 {'1%': -3.430782280885553,
  '5%': -2.8617310495714667,
  '10%': -2.5668716910547014},
 -151765.77746028436)

In [21]:
adfuller((si_val - si_vwap).dropna(), autolag="AIC")

(-10.555203028796987,
 7.985439359068613e-19,
 1,
 15063,
 {'1%': -3.430784203992321,
  '5%': -2.861731899439858,
  '10%': -2.5668721434298236},
 -146838.3888280859)